# OpenAI 빠른 시작

# 개요  
"대규모 언어 모델은 텍스트를 텍스트로 매핑하는 함수입니다. 텍스트 입력 문자열이 주어지면 대규모 언어 모델은 다음에 올 텍스트를 예측하려고 시도합니다"(1). 이 "빠른 시작" 노트북은 사용자에게 고수준의 LLM 개념, AML을 시작하기 위한 핵심 패키지 요구사항, 프롬프트 설계에 대한 간단한 소개, 그리고 다양한 사용 사례의 짧은 예제를 소개합니다.  

더 많은 빠른 시작 예제를 보려면 공식 Azure Open AI 빠른 시작 문서를 참조하세요: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio

## 목차  

[개요](#overview)  
[OpenAI 서비스 사용 방법](#how-to-use-openai-service)  
[1. OpenAI 서비스 생성](#1.-creating-your-openai-service)  
[2. 설치](#2.-installation)    
[3. 자격 증명](#3.-credentials)  

[사용 사례](#use-cases)    
[1. 텍스트 요약](#1.-summarize-text)  
[2. 텍스트 분류](#2.-classify-text)  
[3. 새로운 제품 이름 생성](#3.-generate-new-product-names)  
[4. 분류기 미세 조정](#4.fine-tune-a-classifier)  
[5. 임베딩!](#5.-embeddings!)

[참고자료](#references)

### Azure OpenAI 서비스 시작하기

새로운 고객은 [접근 신청](https://aka.ms/oai/access)을 해야 합니다.  
승인이 완료되면 고객은 Azure 포털에 로그인하여 Azure OpenAI 서비스 리소스를 생성하고 스튜디오를 통해 모델을 실험할 수 있습니다.  

[빠르게 시작하기 위한 훌륭한 리소스](https://techcommunity.microsoft.com/t5/educator-developer-blog/azure-openai-is-now-generally-available/ba-p/3719177 )


### 첫 번째 프롬프트 작성하기  
이 짧은 연습은 OpenAI 모델에 프롬프트를 제출하는 기본적인 작업 "요약"에 대한 간단한 소개를 제공합니다.  

![](images/generative-AI-models-reduced.jpg)  


**단계**:  
1. Python 환경에 OpenAI 라이브러리 설치  
2. 표준 헬퍼 라이브러리를 로드하고 생성한 OpenAI 서비스에 대한 보안 자격 증명을 설정  
3. 작업에 적합한 모델 선택  
4. 모델을 위한 간단한 프롬프트 생성  
5. 모델 API에 요청 제출!

### 1. 헬퍼 라이브러리 가져오기 및 자격 증명 설정

In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True)

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)


In [2]:
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True)

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)


### 2. 적합한 모델 찾기  
GPT-4 (gpt-4o, gpt-4o-xxxx) 모델은 자연어를 이해하고 생성할 수 있습니다. 이 서비스는 네 가지 모델 기능을 제공하며, 각각은 다양한 작업에 적합한 수준의 성능과 속도를 제공합니다. 

[Azure OpenAI 모델](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/models)  
![](images/a-b-c-d-models-reduced.jpg)  


In [3]:
# Select the GPT-4o model for text
model = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

### 3. 프롬프트 설계  

"대규모 언어 모델의 마법은 방대한 양의 텍스트에 대한 예측 오류를 최소화하도록 훈련됨으로써, 모델이 이러한 예측에 유용한 개념을 학습하게 된다는 점입니다. 예를 들어, 모델은 다음과 같은 개념을 학습합니다"(1):

* 철자법
* 문법 작동 방식
* 패러프레이징 방법
* 질문에 답하는 방법
* 대화를 이어가는 방법
* 여러 언어로 글쓰기
* 코딩 방법
* 기타 등등

#### 대규모 언어 모델 제어 방법  
"대규모 언어 모델의 입력 중 가장 영향력이 큰 것은 텍스트 프롬프트입니다(1).

대규모 언어 모델은 다음과 같은 몇 가지 방법으로 출력하도록 프롬프트를 받을 수 있습니다:

명령: 모델에게 원하는 것을 말하기
완성: 원하는 것의 시작 부분을 모델이 완성하도록 유도
데모: 모델에게 원하는 것을 보여주기, 다음 중 하나를 사용하여:
프롬프트에 몇 가지 예제 포함
수백 또는 수천 개의 예제를 포함한 미세 조정 훈련 데이터셋"



#### 프롬프트를 작성하는 세 가지 기본 지침:

**보여주고 말하기**. 지침, 예제 또는 둘의 조합을 통해 원하는 것을 명확히 하세요. 예를 들어, 모델이 항목 목록을 알파벳 순으로 정렬하거나 문단을 감정으로 분류하도록 하려면, 그렇게 하길 원한다는 것을 보여주세요.

**품질 데이터 제공**. 분류기를 구축하거나 패턴을 따르도록 모델을 유도하려는 경우, 충분한 예제가 있는지 확인하세요. 예제를 교정하세요 — 모델은 기본적인 철자 오류를 간파하고 응답을 제공할 만큼 똑똑하지만, 이것이 의도적이라고 가정할 수도 있으며 이는 응답에 영향을 미칠 수 있습니다.

**설정을 확인하세요.** 온도 및 top_p 설정은 모델이 응답을 생성하는 데 얼마나 결정적인지를 제어합니다. 하나의 정답만 있는 응답을 요청하는 경우, 이를 낮게 설정해야 합니다. 더 다양한 응답을 원한다면, 이를 높게 설정해야 합니다. 사람들이 이 설정에서 가장 많이 저지르는 실수는 이를 "영리함" 또는 "창의성" 제어로 간주하는 것입니다.


출처: https://github.com/Azure/OpenAI/blob/main/How%20to/Completions.md

![](images/prompt_design.jpg)
image is creating your first text prompt!

### 4. 실행

In [4]:
# 첫 번째 프롬프트를 한글로 작성합니다.
text_prompt = "경복궁은 어떤 곳인가요?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

"경복궁은 대한민국 서울특별시에 위치한 조선 시대의 대표적인 궁궐입니다. 1395년에 조선 왕조의 태조 이성계에 의해 건립되었으며, 조선 왕조의 법궁(법적으로 정해진 중심 궁궐)으로 사용되었습니다. 경복궁은 '큰 복을 누리는 궁궐'이라는 뜻을 가지고 있으며, 한국 전통 건축의 아름다움과 역사적 가치를 지닌 장소입니다.\n\n경복궁은 여러 차례의 소실과 재건을 거쳤지만, 현재는 주요 전각들이 복원되어 관람객들에게 개방되고 있습니다. 대표적인 건물로는 근정전(왕의 공식 행사와 국무를 보던 장소), 경회루(연회를 열던 누각), 광화문(정문) 등이 있습니다. 또한 경복궁은 한복 체험, 전통 공연 관람 등 다양한 문화 체험 프로그램도 제공하여 많은 관광객들에게 사랑받는 명소입니다.\n\n요약하자면, 경복궁은 한국의 역사와 문화를 대표하는 중요한 궁궐로서, 조선의 왕실 생활과 전통 건축 양식을 체험할 수 있는 장소입니다."

### 동일한 호출을 반복하여 결과를 비교하세요

In [5]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

"경복궁은 대한민국 서울특별시에 위치한 조선 시대의 대표적인 궁궐입니다. 1395년에 조선 왕조의 창건자 이성계에 의해 처음 건립되었으며, 조선 왕조의 법궁(法宮)으로서 왕이 정치와 행정을 집행하던 중심지였습니다. 경복궁은 '큰 복을 누리는 궁궐'이라는 뜻을 가지고 있으며, 화려한 건축양식과 아름다운 자연 경관이 조화를 이루고 있습니다.\n\n역사적으로 임진왜란 때 크게 파괴되었으나 이후 여러 차례 중건되었고, 현대에는 대한민국의 문화유산으로 보존 및 복원 작업이 활발히 이루어지고 있습니다. 현재 경복궁은 관광 명소로도 유명하며, 궁궐 내부에는 근정전, 경회루, 사정전 등 다양한 전각과 왕실 유물이 전시된 박물관이 있습니다. 전통 한복을 입고 방문할 경우 입장료 할인 혜택을 받을 수 있어 많은 사람들이 전통문화를 체험하러 방문합니다."

## 텍스트 요약  
#### 도전 과제  
텍스트 단락 끝에 'tl;dr:'을 추가하여 텍스트를 요약합니다. 모델이 추가 지침 없이 여러 작업을 수행하는 방법을 확인하세요. tl;dr보다 더 설명적인 프롬프트로 실험하여 모델의 동작을 수정하고 원하는 요약을 사용자 정의할 수 있습니다(3).  

최근 연구에서는 대규모 텍스트 코퍼스를 사전 학습한 후 특정 작업에 대해 미세 조정하는 방법으로 많은 NLP 작업 및 벤치마크에서 상당한 성과를 거두었습니다. 일반적으로 아키텍처에서 작업에 구애받지 않지만, 이 방법은 여전히 수천 또는 수만 개의 예제의 작업별 미세 조정 데이터셋이 필요합니다. 반면에 인간은 일반적으로 몇 가지 예제나 간단한 지침만으로 새로운 언어 작업을 수행할 수 있습니다 - 현재의 NLP 시스템이 여전히 크게 어려움을 겪고 있는 부분입니다. 여기에서는 언어 모델을 확장하면 작업에 구애받지 않는 소수의 예제 성능이 크게 향상되며, 때로는 이전 최첨단 미세 조정 접근 방식과 경쟁할 수 있음을 보여줍니다. 



Tl;dr

# 여러 사용 사례에 대한 연습  
1. 텍스트 요약  
2. 텍스트 분류  
3. 새로운 제품 이름 생성
4. 임베딩
5. 분류기 미세 조정

In [6]:
prompt = "대규모 텍스트 말뭉치에 대한 사전 학습 후 특정 작업에 대한 미세 조정을 통해 많은 NLP 작업과 벤치마크에서 상당한 이점을 입증한 바 있습니다. 이 방법은 일반적으로 아키텍처에서 작업에 구애받지 않지만, 여전히 수천 또는 수만 개의 예제로 구성된 작업별 미세 조정 데이터 세트가 필요합니다. 반면, 인간은 일반적으로 몇 가지 예제나 간단한 명령어만으로 새로운 언어 작업을 수행할 수 있지만, 현재의 NLP 시스템에서는 여전히 많은 어려움을 겪고 있습니다. 여기에서는 언어 모델을 확장하면 작업에 구애받지 않고 소수의 예제만으로도 성능이 크게 향상되며, 때로는 이전의 최첨단 미세 조정 접근 방식에 비해 경쟁 우위에 도달할 수도 있음을 보여줍니다.\n\nTl;dr"


In [7]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

'대규모 사전 학습된 언어 모델은 소수의 예제만으로도 다양한 NLP 작업에서 높은 성능을 발휘할 수 있으며, 이는 기존의 대량 데이터 기반 미세 조정보다 효율적이고 경쟁력 있다는 내용입니다.'

## 텍스트 분류  
#### 도전 과제  
추론 시 제공된 카테고리로 항목을 분류합니다. 다음 예제에서는 프롬프트에 카테고리와 분류할 텍스트를 모두 제공합니다(*playground_reference). 

고객 문의: 안녕하세요, 제 노트북 키보드의 키 하나가 최근에 부러져서 교체가 필요합니다:

분류된 카테고리:


In [8]:
prompt = "다음 문의를 다음 중 하나의 카테고리로 분류합니다: [가격, 하드웨어 지원, 소프트웨어 지원]\n\n문의: 안녕하세요, 노트북 키보드의 키 중 하나가 최근에 고장 나서 교체해야 합니다.\n\n분류된 카테고리:"
print(prompt)

다음 문의를 다음 중 하나의 카테고리로 분류합니다: [가격, 하드웨어 지원, 소프트웨어 지원]

문의: 안녕하세요, 노트북 키보드의 키 중 하나가 최근에 고장 나서 교체해야 합니다.

분류된 카테고리:


In [9]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
       model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

'분류된 카테고리: 하드웨어 지원'

## 새로운 제품 이름 생성
#### 도전 과제
예제 단어에서 제품 이름을 생성합니다. 여기에서는 생성할 제품에 대한 정보를 프롬프트에 포함합니다. 또한 유사한 예제를 제공하여 원하는 패턴을 보여줍니다. 우리는 또한 무작위성을 높이고 더 혁신적인 응답을 얻기 위해 온도 값을 높게 설정했습니다.

제품 설명: 가정용 밀크셰이크 제조기
시드 단어: 빠른, 건강한, 컴팩트.
제품 이름: HomeShaker, Fit Shaker, QuickShake, Shake Maker

제품 설명: 모든 발 크기에 맞는 신발 한 켤레.
시드 단어: 적응 가능한, 맞춤형, 옴니핏.

In [10]:
prompt = "제품 설명: 가정용 밀크쉐이크 메이커\n시드워드: 빠르고, 건강하고, 컴팩트한 제품입니다.\n제품 이름: 홈쉐이커, 핏 쉐이커, 퀵쉐이크, 쉐이크 메이커\n\n제품 설명: 모든 발 사이즈에 맞는 신발\n씨드워드: 적응성, 핏, 옴니핏."

print(prompt)

제품 설명: 가정용 밀크쉐이크 메이커
시드워드: 빠르고, 건강하고, 컴팩트한 제품입니다.
제품 이름: 홈쉐이커, 핏 쉐이커, 퀵쉐이크, 쉐이크 메이커

제품 설명: 모든 발 사이즈에 맞는 신발
씨드워드: 적응성, 핏, 옴니핏.


In [11]:
#Setting a few additional, typical parameters during API Call
response = client.chat.completions.create(
       model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},]
)

response.choices[0].message.content

'제품 설명과 시드워드를 기반으로 제품 이름 제안을 드립니다.\n\n1. 가정용 밀크쉐이크 메이커  \n- 씨드워드: 빠르고, 건강하고, 컴팩트  \n- 이름 제안:  \n  - 홈쉐이커  \n  - 핏쉐이커  \n  - 퀵쉐이크  \n  - 쉐이크 메이커\n\n2. 모든 발 사이즈에 맞는 신발  \n- 씨드워드: 적응성, 핏, 옴니핏  \n- 이름 제안:  \n  - 옴니핏 슈즈  \n  - 에버핏  \n  - 어댑트슈  \n  - 핏올슈즈  \n  - 어댑티브 플렉스\n\n필요하시면 추가 이름 제안이나 마케팅 문구도 도와드릴 수 있습니다.'

## 임베딩!  
이 섹션에서는 임베딩을 검색하고 단어, 문장 및 문서 간의 유사성을 찾는 방법을 보여줍니다

### 모델 분류 - 유사성 모델 선택
가장 강력한 모델(davinci)을 사용하여 유사성 모델을 선택합니다

**모델 분류**: {family} - {capability} - {input-type} - {identifier}  

{family}     --> text-similarity  (일반 텍스트 GPT-3 모델)  
{capability} --> davinci         (ada-babbage-curie-davinci 가족 중 두 번째로 강력한 모델)  
{input-type} --> n/a              (검색 모델에만 지정됨)  
{identifier} --> 001              (버전 001)  

model = 'text-embedding-ada-002'

In [12]:
import numpy as np
model=os.getenv("EMBEDDING_MODEL_NAME")
def cosine_similarity(query_embedding, embeddings, distance_metric='cosine'):
    if distance_metric == 'cosine':
        distances = np.dot(embeddings, query_embedding) / (np.linalg.norm(embeddings) * np.linalg.norm(query_embedding))
        distances = 1 - distances  
    else:
        raise ValueError("Métrica de distância não suportada. Utilize 'cosine'.")

    return distances

In [13]:
text = '게으른 개를 뛰어넘은 재빠른 갈색 여우'
client.embeddings.create(input=[text], model=model).data[0].embedding


[-0.006658022757619619,
 0.005868293810635805,
 0.00044004610390402377,
 -0.02128623239696026,
 0.04104160517454147,
 0.010430497117340565,
 -0.019779672846198082,
 -0.0049054319970309734,
 -0.03316861763596535,
 -0.03039848990738392,
 -0.058221250772476196,
 -0.00023027433780953288,
 0.024177856743335724,
 -0.03734810650348663,
 -0.0036084537860006094,
 0.05073704943060875,
 -0.029693808406591415,
 -0.00422505009919405,
 -0.007429527118802071,
 -0.007672520820051432,
 0.02063015103340149,
 -0.028211548924446106,
 0.048841699957847595,
 -0.01190060842782259,
 -0.024056360125541687,
 0.023728318512439728,
 0.0032287766225636005,
 0.015794578939676285,
 -0.006645873188972473,
 0.005679973866790533,
 0.015575884841382504,
 0.038563072681427,
 -0.01749553345143795,
 0.020824545994400978,
 -0.004130890127271414,
 0.015831029042601585,
 0.03773689642548561,
 0.024882536381483078,
 0.015539436601102352,
 -0.013656236231327057,
 -0.013874930329620838,
 0.02149277739226818,
 -0.0368378199636936

In [14]:
# compare several words
automobile_embedding    = client.embeddings.create(input='자동차', model=model).data[0].embedding
vehicle_embedding       = client.embeddings.create(input='차량', model=model).data[0].embedding
dinosaur_embedding      = client.embeddings.create(input='공룡', model=model).data[0].embedding
stick_embedding         = client.embeddings.create(input='스틱', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

0.3991128479510532
0.7223092337004864
0.7344895658606448


## cnn daily news 데이터셋의 기사 비교
출처: https://huggingface.co/datasets/cnn_dailymail


In [15]:
import pandas as pd

# 번역된 기사와 하이라이트

cnn_daily_articles = [
    "브레멘, 독일 -- 2004년 FC 포르투가 모나코를 꺾고 챔피언스리그 결승에서 우승할 때 골을 넣었던 카를로스 알베르토가 분데스리가 클럽 베르더 브레멘에 구단 역대 최고 이적료인 780만 유로(1,070만 달러)에 합류했습니다. 카를로스 알베르토는 조제 무리뉴 감독 아래 FC 포르투에서 성공을 거뒀습니다. '나는 베르더와 함께 우승하기 위해 여기 왔습니다.'라고 22세의 그는 새 클럽에서 첫 훈련을 마친 뒤 말했습니다. '나는 브레멘이 마음에 들고, 오직 여기만 오고 싶었습니다.' 카를로스 알베르토는 플루미넨시에서 커리어를 시작해 2002년 캄페오나토 카리오카 우승을 도왔습니다. 2004년 1월에는 조제 무리뉴 감독이 이끌던 FC 포르투로 이적해 포르투갈 리그와 챔피언스리그 우승을 차지했습니다. 2005년 초에는 코린치안스로 이적해 브라질 세리 A 우승에 기여했으나, 2006년 코린치안스가 부진하자 에메르손 레앙 감독과 불화가 생겼습니다. 이들의 관계는 코파 수다메리카나에서 클럽 아틀레티코 라누스와의 경기에서 절정에 달했고, 카를로스 알베르토는 레앙 감독이 있는 한 다시는 코린치안스에서 뛰지 않겠다고 선언했습니다. 올해 1월부터는 친정팀 플루미넨시에서 임대 생활을 했습니다. 분데스리가 챔피언인 슈투트가르트는 일요일 레알 사라고사에서 에베르톤을 임대로 영입할 것이라고 밝혔습니다. 에베르톤은 2001~2005년 보루시아 도르트문트에서 활약한 바 있습니다. 금요일에는 2004년 분데스리가 득점왕이었던 아일톤이 레드스타 베오그라드에서 뒤스부르크로 1년 계약을 맺고 독일로 복귀했습니다. 친구에게 이메일 보내기.",
    "(CNN) -- 축구 슈퍼스타, 셀러브리티, 패션 아이콘, 수백만 달러의 인기남. 이제 데이비드 베컴이 미국 메이저리그 사커에서 활약하기 위해 할리우드 힐스로 향합니다. CNN은 베컴이 맨체스터 유나이티드에서 뛰는 꿈을 어떻게 이뤘는지, 그리고 잉글랜드 대표팀에서의 시간을 조명합니다. 세계적으로 유명한 축구선수 베컴은 LA 갤럭시와 5년 계약을 맺었고, 금요일에는 기자회견을 열고 새로운 등번호를 공개할 예정입니다. 이번 주, CNN의 '벡스' 베키 앤더슨이 베컴의 축구선수, 패션 아이콘, 글로벌 현상으로서의 삶을 심층적으로 살펴봅니다. 동런던 거리에서 할리우드 힐스까지, 베컴의 놀라운 성공 여정을 따라갑니다. 그녀는 미국 스포츠/연예계에서 가장 뜨거운 인물인 베컴의 진면목을 파헤치고, 그를 움직이게 하는 동기와 '황금발'의 비밀을 탐구합니다. CNN은 맨유에서의 꿈, 팝스타 빅토리아와의 결혼, 잉글랜드 대표팀에서의 고난과 영광, 레알 마드리드 이적, 그리고 이제 LA 홈디포 스타디움까지 베컴의 인생을 돌아봅니다. 베컴과 가족이 LA 생활에 어떻게 적응할지, 현지인들과 명소, 셀럽 문화 적응기를 다룹니다. 베컴은 이미 광고, 게임, 패션 등 다양한 분야에서 얼굴을 알렸습니다. 미국에서 축구가 '여자아이들만의 스포츠'라는 인식이 바뀌고 있으며, 점점 더 많은 아이들이 유럽 축구를 선택하고 있습니다. CNN은 미국에서 활약한 해외 스타들의 영향과 현재의 변화를 살펴보고, LA의 데이비드 베컴 아카데미도 조명합니다. 친구에게 이메일 보내기.",
    "로스앤젤레스, 캘리포니아(CNN) -- 이라크에서 화상을 입은 5살 소년 유시프가 유니버설 스튜디오에서 코너를 돌자마자 가장 좋아하는 슈퍼히어로를 만났습니다. 유시프는 스파이더맨의 열렬한 팬이었습니다. '가장 좋았어요.'라고 그는 말했습니다. 스파이더맨은 다른 슈퍼히어로들과 함께 사륜 오토바이를 타고 등장했습니다. 스파이더맨은 유시프에게 다가와 인사를 건네고, 상징적인 파란색과 빨간색 타이즈로 소년을 부드럽게 안아주었습니다. 그는 유시프에게 손목에서 거미줄 쏘는 법 등 몇 가지 묘기를 보여줬습니다. 이번엔 진짜 거미줄은 나오지 않았지만요. '좋았어, 유시프!' 스파이더맨이 소년이 손동작을 따라하자 말했습니다. 다른 슈퍼히어로들도 몰려와 구경했습니다. 그린 고블린도 소년에게 인사를 건넸습니다. 유시프는 악당에게는 별로 관심이 없었습니다. 스파이더맨이 최고였죠. '가장 좋았어요.'라고 소년은 나중에 다시 말했습니다. '다시 만나고 싶어요.' 그리고 덧붙였습니다. '진짜 스파이더맨이 아니라는 건 알아요.' 이날은 소년의 악몽이 잠시 잊힌 꿈같은 하루였습니다. 그는 스폰지밥, 래시, 3살 오랑우탄 아치도 만났습니다. 아치는 유시프의 손을 잡고 놓지 않았습니다. 유시프가 손을 빼도 다시 잡으려 했습니다. 유시프는 놀이방에서 스펀지볼을 쏘며 깔깔 웃었습니다. 바그다드에서 보던 무기와는 전혀 다른 장난감이었습니다. 그는 트램을 타고 유니버설 스튜디오의 백스테이지도 돌았습니다. 한순간 차가 흔들리고, 불과 연기가 피어오르고, 트럭이 돌진해왔지만 가족은 무사했습니다. '난 무서웠다.' 아빠가 말했습니다. '난 안 무서웠어요.' 유시프가 대답했습니다. 부모님은 하루 종일 미소를 지었습니다. 유시프는 14개월 된 여동생 아야를 유모차에 태워 밀었습니다. '우리가 여기 오고 싶었는지 물어볼 필요가 있었나요?' 아빠는 감탄했습니다. '결혼식 빼고 오늘이 내 인생에서 가장 행복한 날이에요.' 하루 전, 부모는 이라크에서 미국으로 오게 된 사연과 9개월 전 복면을 쓴 남자들이 집 앞에서 아들을 납치해 불을 지른 일을 이야기했습니다. 엄마는 집 안에서 아들의 비명을 들었고, 아빠는 바그다드 전역을 돌며 도움을 구했지만 아무도 도와주지 않았습니다. 두 달간의 입원, 마취도 없이 치료받는 아들의 비명을 병원 밖에서 들었습니다. CNN에 사연을 알리는 것이 가족의 생명을 위협할 수 있다는 걸 알았지만, 아들을 위해서라면 뭐든지 하겠다고 했습니다. '이라크는 끝났다.' 아빠는 영어로 말했습니다. 아랍어로는 조국이 이런 자유를 누릴 수 없을 거라고 덧붙였습니다. 너무 많은 폭력과 살인 때문입니다. 두 아이 모두 전쟁만 보고 자랐지만, 이번 주 미국에서의 삶은 전혀 달랐습니다. '꿈만 같아요.' 아빠는 말했습니다. 그는 바그다드에서 자원봉사를 많이 했다고 했습니다. '아마 그래서 지금 도움을 받는 것 같아요.' 유니버설 스튜디오에서 아빠는 계곡을 내려다보며 '좋은 미국, 좋은 미국'이라고 영어로 말했습니다. 친구에게 이메일 보내기. CNN의 아르와 데이먼이 이 보도에 기여했습니다."
]

cnn_daily_article_highlights = [
    "베르더 브레멘, 카를로스 알베르토 영입에 구단 최고 이적료 1,070만 달러 지불.\n브라질 미드필더, 2004년 FC 포르투와 챔피언스리그 우승.\n올해 1월부터 친정팀 플루미넨시에서 임대 생활.",
    "베컴, LA 갤럭시와 5년 계약 체결.\n새 계약은 2007년 7월 1일부터 발효.\n전 잉글랜드 주장, 금요일 기자회견 및 새 등번호 공개 예정.\nCNN, 축구선수·패션 아이콘·글로벌 현상으로서의 베컴 조명.",
    '소년, 스파이더맨 만남에 "가장 좋았어요"\n유시프, 유니버설 스튜디오에서 스폰지밥·래시·오랑우탄도 만남.\n아빠: "결혼식 빼고 오늘이 내 인생에서 가장 행복한 날"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()
                   

,articles,highligths
0,"브레멘, 독일 -- 2004년 FC 포르투가 모나코를 꺾고 챔피언스리그 결승에서 우...","베르더 브레멘, 카를로스 알베르토 영입에 구단 최고 이적료 1,070만 달러 지불...."
1,"(CNN) -- 축구 슈퍼스타, 셀러브리티, 패션 아이콘, 수백만 달러의 인기남. ...","베컴, LA 갤럭시와 5년 계약 체결.\n새 계약은 2007년 7월 1일부터 발효...."
2,"로스앤젤레스, 캘리포니아(CNN) -- 이라크에서 화상을 입은 5살 소년 유시프가 ...","소년, 스파이더맨 만남에 ""가장 좋았어요""\n유시프, 유니버설 스튜디오에서 스폰지밥..."


In [16]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

0.6086936753931762
0.8596084049407773


## 꼬맨틀

https://semantle-ko.newsjel.ly/

임베딩 모델을 이용하여 위와 유사한 게임 만들기

# 참고자료  
-Azure Reference Documentation  
-Azure OpenAI GitHub Repo
-cookbooks  
-OpenAI website  

1 - [Openai Cookbook](https://github.com/openai/openai-cookbook)  
2 - [Azure Documentation - Azure Open AI Models](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/models)  
